In [1]:
import json
import pickle

import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib.cm import get_cmap
import numpy as np
import open3d as o3d
import numpy as np
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
import open3d as o3d

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation, scan3r

data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_number = "000007"
curr_frame_number = "000007"
new_frame_number = "000008"
patch_h= 18
image_height = 540
image_width = 960
patch_w = 32
patch_height = 30
patch_width = 30

# path = osp.join(data_dir, "files", "val_scans.txt")
# # Initialize an empty list to store UUIDs
# uuids = []

# # Open the file in read mode
# with open(path, 'r') as file:
#     # Iterate over each line in the file
#     for line in file:
#         # Strip leading and trailing whitespace (including newlines)
#         line = line.strip()
#         # Append the UUID to the list
#         uuids.append(line)


# path2 = osp.join(data_dir, "files", "train_resplit_scans.txt")
# # Initialize an empty list to store UUIDs
# uuids2 = []

# # Open the file in read mode
# with open(path2, 'r') as file:
#     # Iterate over each line in the file
#     for line in file:
#         # Strip leading and trailing whitespace (including newlines)
#         line = line.strip()
#         # Append the UUID to the list
#         uuids2.append(line)





# # Print the list of UUIDs
# for i in uuids:
#     if i in uuids2:
#         print("is in the list")

# print("done")


# import shutil

# # Define the path to the parent folder
# parent_folder = osp.join(data_dir, "scenes")

# path = osp.join(data_dir, "files", "to_be_deleted.txt")
# # Initialize an empty list to store UUIDs
# directories_to_delete = []

# # Open the file in read mode
# with open(path, 'r') as file:
    # Iterate over each line in the file
#     for line in file:
#         # Strip leading and trailing whitespace (including newlines)
#         line = line.strip()
#         # Append the UUID to the list
#         directories_to_delete.append(line)

# scan_info_file = osp.join(data_dir, "files", '3RScan.json')
# file = open(scan_info_file)
# data = json.load(file)
# file.close()
# refscans2scans = {}
# scans2refscans = {}
# all_scans_split = []
# for scan_data in data:
#     ref_scan_id = scan_data['reference']
#     refscans2scans[ref_scan_id] = [ref_scan_id]
#     scans2refscans[ref_scan_id] = ref_scan_id
#     for scan in scan_data['scans']:
#             refscans2scans[ref_scan_id].append(scan['reference'])
#             scans2refscans[scan['reference']] = ref_scan_id


# all_scans_split = []
#     ## get all scans within the split(ref_scan + rescan)
# for ref_scan in directories_to_delete:
#     all_scans_split += refscans2scans[ref_scan]

# scan_ids = all_scans_split


# print("scanids", len(scan_ids))
# print("parent folder", parent_folder)
# # Iterate over the list and delete each directory
# for dir_name in scan_ids:
#     dir_path = os.path.join(parent_folder, dir_name)
    
#     # Check if the directory exists
#     if os.path.exists(dir_path):
#         # Delete the directory and all its contents
#         shutil.rmtree(dir_path)
#         print(f"Deleted directory: {dir_path}")
#     else:
#         print(f"Directory not found: {dir_path}")

# print("Completed directory deletion.")







Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/local/home/ekoller/BT
scanids 314
parent folder /local/home/ekoller/R3Scan/scenes
Deleted directory: /local/home/ekoller/R3Scan/scenes/0988ea72-eb32-2e61-8344-99e2283c2728
Deleted directory: /local/home/ekoller/R3Scan/scenes/9766cbe5-6321-2e2f-8040-4e5b7a5d8ba1
Deleted directory: /local/home/ekoller/R3Scan/scenes/9766cbf5-6321-2e2f-8131-78c4e204635d
Deleted directory: /local/home/ekoller/R3Scan/scenes/9766cbf7-6321-2e2f-81e1-2b46533c64dd
Deleted directory: /local/home/ekoller/R3Scan/scenes/4238490c-60a7-271e-9f38-3c651e3b3912
Deleted directory: /local/home/ekoller/R3Scan/scenes/4238490a-60a7-271e-9c04-3846221dc354
Deleted directory: /local/home/ekoller/R3Scan/scenes/42384908-60a7-271e-9c46-01e562c8974c
Deleted directory: /local/home/ekoller/R3Scan/scenes/10b17963-3938-2467-8a48-0d4af350ce92
Deleted directory: /local/ho

the point of this notebook is to create a visualization of the data in a vizualizable space


In [8]:
#for a given scene get the colours of the differnt object_ids so that we can colour the different objects
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color[::-1])
    
    return object_colors

colors = get_id_colours(data_dir,curr_scan_id)
print("cur scan id colors", colors.keys())
print("colour obj 4", tuple(c / 255.0 for c in colors[4]))



cur scan id colors dict_keys([4, 18, 2, 0, 3, 1, 100, 16, 8, 9, 24, 25, 29, 6, 26, 10, 11, 23, 27, 22, 5, 20, 12, 15, 13, 14, 103, 21])
colour obj 4 (0.054901960784313725, 0.4980392156862745, 1.0)


In [10]:
#some paths which will get used 
#path to the projcetion features - all of them
all_info_path = osp.join(data_dir,"files/Features2D/projection/DinoV2/patch_32_18", curr_scan_id + ".pkl")
with open(all_info_path, 'rb') as file:
    all_features_curr_scene = pickle.load(file)


#path to the projcetion features - top 15
top_15_info_path = osp.join(data_dir,"files/Features2D/top_15_projection/DinoV2/patch_32_18", curr_scan_id + ".pkl")
with open(top_15_info_path, 'rb') as file:
    top_15_curr_scene = pickle.load(file)


compute and visualize the PCA of all the feature vectors


In [12]:
#pca for all feature vectors, take every vector, perform a mean aggregation and then put them together
all_features = []
all_colors = []

print("starting to collect matricies")
#get every fearure vector and average the patches to one patch
for frame in all_features_curr_scene:
    for obj_id in all_features_curr_scene[frame]:
        if obj_id != 0:
            matrix = all_features_curr_scene[frame][obj_id]

            aggregated_feature = np.mean(matrix, axis=0)  # Mean across patches
            all_features.append(aggregated_feature)

            all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#perform max pooling instead of averaging
# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Apply max pooling across patches
#             aggregated_feature = np.max(matrix, axis=0)  # Max pooling across patches
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#concatinate all the patches to keep all the info

# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Concatenate all patches into a single vector
#             aggregated_feature = matrix.flatten()  # Flatten the matrix into a single long vector
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))
        
      
print("finished collecting matricies")


# Convert list to numpy array
all_features = np.array(all_features)
all_colors = np.array(all_colors)

# Apply PCA to the aggregated feature vectors
pca = PCA(n_components=3)
reduced_points = pca.fit_transform(all_features)

# Create a point cloud for visualization
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(reduced_points)
point_cloud.colors = o3d.utility.Vector3dVector(all_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud], window_name='3D PCA Visualization', width=800, height=600)





starting to collect matricies
finished collecting matricies


In [52]:
#pca for top_15 vectors, take every vector, perform a mean aggregation and then put them together
all_features = []
all_colors = []

print("starting to collect matricies")
#get every fearure vector and average the patches to one patch
for obj_id in top_15_curr_scene['obj_visual_emb']:
    for frame in top_15_curr_scene['obj_visual_emb'][obj_id]:
        if obj_id != 0:
            matrix = top_15_curr_scene['obj_visual_emb'][obj_id][frame]
            aggregated_feature = np.mean(matrix, axis=0)  # Mean across patches
            all_features.append(aggregated_feature)

            all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))
        
      
print("finished collecting matricies")


# Convert list to numpy array
all_features = np.array(all_features)
all_colors = np.array(all_colors)

# Apply PCA to the aggregated feature vectors
pca = PCA(n_components=3)
reduced_points = pca.fit_transform(all_features)

# Create a point cloud for visualization
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(reduced_points)
point_cloud.colors = o3d.utility.Vector3dVector(all_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud], window_name='3D PCA Visualization', width=800, height=600)

starting to collect matricies
finished collecting matricies


the next part performs mds

In [86]:
import numpy as np
from sklearn.manifold import MDS
import open3d as o3d

# Initialize lists to store features and colors
all_features = []
all_colors = []

print("Starting to collect matrices")

#get every fearure vector and average the patches to one patch
for frame in all_features_curr_scene:
    for obj_id in all_features_curr_scene[frame]:
        if obj_id != 0:
            matrix = all_features_curr_scene[frame][obj_id]

            aggregated_feature = np.mean(matrix, axis=0)  # Mean across patches
            all_features.append(aggregated_feature)

            all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#perform max pooling instead of averaging
# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Apply max pooling across patches
#             aggregated_feature = np.max(matrix, axis=0)  # Max pooling across patches
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#concatinate all the patches to keep all the info

# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Concatenate all patches into a single vector
#             aggregated_feature = matrix.flatten()  # Flatten the matrix into a single long vector
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

print("Finished collecting matrices")

# Convert lists to numpy arrays
all_features = np.array(all_features)
all_colors = np.array(all_colors)

# Apply MDS to the aggregated feature vectors
mds = MDS(n_components=3, random_state=42)
reduced_points = mds.fit_transform(all_features)

# Create a point cloud for visualization
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(reduced_points)
point_cloud.colors = o3d.utility.Vector3dVector(all_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud], window_name='3D MDS Visualization', width=800, height=600)


Starting to collect matrices
Finished collecting matrices


the next part does t-sne


In [80]:
import numpy as np
from sklearn.manifold import TSNE
import open3d as o3d

# Initialize lists to store features and colors
all_features = []
all_colors = []

print("Starting to collect matrices")

#get every fearure vector and average the patches to one patch
for frame in all_features_curr_scene:
    for obj_id in all_features_curr_scene[frame]:
        if obj_id != 0:
            matrix = all_features_curr_scene[frame][obj_id]

            aggregated_feature = np.mean(matrix, axis=0)  # Mean across patches
            all_features.append(aggregated_feature)

            all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#perform max pooling instead of averaging
# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Apply max pooling across patches
#             aggregated_feature = np.max(matrix, axis=0)  # Max pooling across patches
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

#concatinate all the patches to keep all the info

# for frame in all_features_curr_scene:
#     for obj_id in all_features_curr_scene[frame]:
#         if obj_id != 0:
#             matrix = all_features_curr_scene[frame][obj_id]

#             # Concatenate all patches into a single vector
#             aggregated_feature = matrix.flatten()  # Flatten the matrix into a single long vector
#             all_features.append(aggregated_feature)

#             # Append color corresponding to the object id
#             all_colors.append(tuple(c / 255.0 for c in colors[obj_id]))

print("Finished collecting matrices")

# Convert lists to numpy arrays
all_features = np.array(all_features)
all_colors = np.array(all_colors)

# Apply t-SNE to the aggregated feature vectors
tsne = TSNE(n_components=3, random_state=42)
reduced_points = tsne.fit_transform(all_features)

# Create a point cloud for visualization
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(reduced_points)
point_cloud.colors = o3d.utility.Vector3dVector(all_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud], window_name='3D t-SNE Visualization', width=1600, height=1200)


Starting to collect matrices
Finished collecting matrices


/local/home/ekoller/anaconda3/envs/BT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/local/home/ekoller/anaconda3/envs/BT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [8]:
# Inspect the loaded data
def inspect_data(data, indent=0):
    space = " " * indent
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{space}{key}:")
            inspect_data(value, indent + 2)
    elif isinstance(data, list):
        print(f"{space}List with {len(data)} elements")
        if len(data) > 0 and not isinstance(data[0], (dict, list)):
            print(f"{space}{data[:3]}...")  # Show only the first 3 elements
        else:
            for i, item in enumerate(data):
                print(f"{space}[{i}]")
                inspect_data(item, indent + 2)
    else:
        print(f"{space}{data}")


#get the data information on the features input of the scene
input_info_path = osp.join(data_dir,"files/Features2D/projection/DinoV2/patch_32_18", curr_scan_id + ".pkl")
with open(input_info_path, 'rb') as file:
    input_info = pickle.load(file)
    
#inspect_data(input_info)

#we get: frame: object: feaurevector



#get the Dinov2 features for the input image ( the segmented one)
input_featues_path = osp.join(data_dir,"files/Features2D/top_15_projection/DinoV2/patch_32_18", curr_scan_id + ".pkl")
with open(input_featues_path, 'rb') as file:
    input_features = pickle.load(file)

inspect_data(input_features)

#we get 'obj_visual_emb': obj_visual_emb: objectid, frame id, feautre vector   ; 'obj_image_votes_topK': list of the top k frame numbers
    

    

obj_visual_emb:
  4:
    000013:
      [[-0.01427012 -0.04461832  0.02815611 ...  0.03093896 -0.02517844
  -0.04104724]
 [-0.02760206 -0.04691809  0.03423353 ...  0.01666715  0.00951616
  -0.03162487]
 [-0.02824055 -0.04314344  0.03374052 ...  0.01605237  0.0078432
  -0.04279102]
 ...
 [-0.02186038 -0.03891225 -0.03601225 ... -0.00804644 -0.01346326
   0.00735536]
 [-0.02189068 -0.04871441 -0.02200015 ...  0.00865211  0.00317558
   0.01607553]
 [-0.01747567 -0.01156992 -0.03005073 ...  0.00782268 -0.03662089
   0.01947295]]
    000027:
      [[ 0.00525972 -0.02053129 -0.00878008 ... -0.00857964  0.01594808
  -0.01750733]
 [ 0.00617599 -0.01502265  0.00723979 ... -0.00978271 -0.00193295
  -0.00949512]
 [ 0.00173849 -0.02345072 -0.00456836 ... -0.00413504  0.0257508
  -0.02845413]
 ...
 [-0.02238576 -0.03204476 -0.03840893 ... -0.01578025 -0.02549137
   0.04364133]
 [-0.00551306 -0.02721707 -0.00163641 ... -0.01291007 -0.03780796
   0.04271993]
 [-0.02860978 -0.0335728  -0.0362578  ...  